## Bank Customer Purchase Forecast
Data Source: https://tianchi.aliyun.com/competition/entrance/531993/information

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier

In [2]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [3]:
X_train = data_train.drop(["id", "subscribe"], axis = 1)
y_train = data_train["subscribe"].replace(["yes", "no"], [1, 0])
X_test = data_test.drop("id", axis = 1)

In [4]:
X_train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed
0,51,admin.,divorced,professional.course,no,yes,yes,cellular,aug,mon,4621,1,112,2,failure,1.4,90.81,-35.53,0.69,5219.74
1,50,services,married,high.school,unknown,yes,no,cellular,may,mon,4715,1,412,2,nonexistent,-1.8,96.33,-40.58,4.05,4974.79
2,48,blue-collar,divorced,basic.9y,no,no,no,cellular,apr,wed,171,0,1027,1,failure,-1.8,96.33,-44.74,1.50,5022.61
3,26,entrepreneur,single,high.school,yes,yes,yes,cellular,aug,fri,359,26,998,0,nonexistent,1.4,97.08,-35.55,5.11,5222.87
4,45,admin.,single,university.degree,no,no,no,cellular,nov,tue,3178,1,240,4,success,-3.4,89.82,-33.83,1.17,4884.70


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               22500 non-null  int64  
 1   job               22500 non-null  object 
 2   marital           22500 non-null  object 
 3   education         22500 non-null  object 
 4   default           22500 non-null  object 
 5   housing           22500 non-null  object 
 6   loan              22500 non-null  object 
 7   contact           22500 non-null  object 
 8   month             22500 non-null  object 
 9   day_of_week       22500 non-null  object 
 10  duration          22500 non-null  int64  
 11  campaign          22500 non-null  int64  
 12  pdays             22500 non-null  int64  
 13  previous          22500 non-null  int64  
 14  poutcome          22500 non-null  object 
 15  emp_var_rate      22500 non-null  float64
 16  cons_price_index  22500 non-null  float6

In [6]:
data_train.isnull().sum()

id                  0
age                 0
job                 0
marital             0
education           0
default             0
housing             0
loan                0
contact             0
month               0
day_of_week         0
duration            0
campaign            0
pdays               0
previous            0
poutcome            0
emp_var_rate        0
cons_price_index    0
cons_conf_index     0
lending_rate3m      0
nr_employed         0
subscribe           0
dtype: int64

In [7]:
data_train.select_dtypes(exclude = "number").nunique(axis = 0)

job            12
marital         4
education       8
default         3
housing         3
loan            3
contact         2
month          10
day_of_week     5
poutcome        3
subscribe       2
dtype: int64

In [8]:
data_train["job"].unique()
for col in data_train.select_dtypes(exclude = "number").columns:
    print(col, ": ", data_train[col].unique())

job :  ['admin.' 'services' 'blue-collar' 'entrepreneur' 'management'
 'technician' 'housemaid' 'self-employed' 'unemployed' 'retired' 'student'
 'unknown']
marital :  ['divorced' 'married' 'single' 'unknown']
education :  ['professional.course' 'high.school' 'basic.9y' 'university.degree'
 'unknown' 'basic.4y' 'basic.6y' 'illiterate']
default :  ['no' 'unknown' 'yes']
housing :  ['yes' 'no' 'unknown']
loan :  ['yes' 'no' 'unknown']
contact :  ['cellular' 'telephone']
month :  ['aug' 'may' 'apr' 'nov' 'jul' 'jun' 'oct' 'dec' 'sep' 'mar']
day_of_week :  ['mon' 'wed' 'fri' 'tue' 'thu']
poutcome :  ['failure' 'nonexistent' 'success']
subscribe :  ['no' 'yes']


In [9]:
data_train.describe()

,id,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed
count,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000,22500.000000
mean,11250.500000,40.407511,1146.303733,3.364800,773.991956,1.316444,0.078529,93.548785,-39.877180,3.302490,5137.211285
std,6495.334864,12.086078,1432.432125,7.223837,326.934334,1.918733,1.573831,2.805786,5.805442,1.611777,170.670611
min,1.000000,16.000000,0.000000,0.000000,0.000000,0.000000,-3.400000,87.640000,-53.280000,0.600000,4715.420000
25%,5625.750000,32.000000,143.000000,1.000000,557.750000,0.000000,-1.800000,91.190000,-44.160000,1.430000,5008.510000
50%,11250.500000,38.000000,353.000000,1.000000,964.000000,0.000000,1.100000,93.540000,-40.600000,3.920000,5133.955000
75%,16875.250000,47.000000,1873.000000,3.000000,1005.000000,2.000000,1.400000,95.920000,-35.797500,4.830000,5267.677500
max,22500.000000,101.000000,5149.000000,57.000000,1048.000000,6.000000,1.400000,99.460000,-25.550000,5.270000,5489.500000


In [10]:
data_train.at[(data_train.pdays > 0) & (data_train.previous == 0), "previous"] = 1

In [11]:
data_train = data_train.replace("unknown", pd.NA)
data_test = data_test.replace("unknown", pd.NA)

data_train[["duration", "campaign", "pdays", "previous"]] = data_train[["duration", "campaign", "pdays", "previous"]].replace(0, pd.NA)
data_test[["duration", "campaign", "pdays", "previous"]] = data_test[["duration", "campaign", "pdays", "previous"]].replace(0, pd.NA)

In [12]:
data_train.isnull().sum()

id                     0
age                    0
job                  274
marital              320
education            994
default             4860
housing              886
loan                 889
contact                0
month                  0
day_of_week            0
duration               2
campaign            4480
pdays                 19
previous               1
poutcome               0
emp_var_rate           0
cons_price_index       0
cons_conf_index        0
lending_rate3m         0
nr_employed            0
subscribe              0
dtype: int64

In [13]:
num_proc = make_pipeline(StandardScaler(), KNNImputer(n_neighbors=50, weights = "distance"))#KNNImputer(), 
cat_proc = make_pipeline(SimpleImputer(strategy = "most_frequent"), OneHotEncoder(handle_unknown = "ignore"))

preprocessor = make_column_transformer((cat_proc, X_train.select_dtypes(exclude = "number").columns), 
                                      (num_proc, X_train.select_dtypes(include = "number").columns))

In [14]:
model_xgb = make_pipeline(preprocessor,
                         XGBClassifier(n_estimators = 2000, #2000
                                       max_depth = 5, #5
                                       #min_weight_fraction_leaf = 0.1,
                                       subsample = 0.95, 
                                       learning_rate = 0.01,
                                       objective='binary:logistic'))

model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)

In [15]:
n_cv_folds = 5
cv_result = cross_validate(model_xgb, X_train, y_train, cv = n_cv_folds, scoring = "accuracy")
cv_result["test_score"].mean()

0.8844000000000001

In [16]:
result = pd.DataFrame({"id": data_test["id"], "subscribe": y_pred})

In [17]:
result["subscribe"].replace([1, 0], ["yes", "no"], inplace = True)

In [18]:
result.to_csv("submission.csv", index = False)

Accuracy on the test dataset: 0.9663